## as the last model has 0.934 train accuracy and 0.734 test accuracy it is clearly overfitting so I'll try regulazation first then drop out next

## implement and solve Vanishing / Exploding Gradients

In [ ]:
from dnn_structure import *

import numpy as np
import sys
sys.path.append("../src")
from Load_data.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from Load_data.data_loader import load_and_resize_images, build_label_array


In [ ]:

TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000



labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

## L2 Regularization

In [ ]:
def compute_cost_with_regularization(A4, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization.
    
    Arguments:
    A4 -- output of forward propagation (last activation), shape (1, m)
    Y -- true labels, shape (1, m)
    parameters -- dictionary containing W1, W2, W3, W4
    lambd -- regularization parameter
    
    Returns:
    cost -- regularized cost
    """
    m = Y.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    W4 = parameters["W4"]
    
    # Cross-entropy cost
    cross_entropy_cost = compute_cost(A4, Y)
    
    # L2 regularization cost
    L2_regularization_cost = (lambd / (2 * m)) * (
        np.sum(np.square(W1)) +
        np.sum(np.square(W2)) +
        np.sum(np.square(W3)) +
        np.sum(np.square(W4))
    )
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost


In [ ]:
def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements backward propagation with L2 regularization
    
    Arguments:
    X -- input data, shape (input size, m)
    Y -- true labels, shape (1, m)
    cache -- output from forward propagation
    lambd -- regularization parameter
    
    Returns:
    gradients -- dictionary of gradients
    """
    
    m = X.shape[1]
    
    (Z1, A1, W1, b1,
     Z2, A2, W2, b2,
     Z3, A3, W3, b3,
     Z4, A4, W4, b4) = cache
     
    # OUTPUT LAYER
    dZ4 = A4 - Y
    dW4 = 1./m * np.dot(dZ4, A3.T) + (lambd/m) * W4
    db4 = 1./m * np.sum(dZ4, axis=1, keepdims=True)
    
    # LAYER 3
    dA3 = np.dot(W4.T, dZ4)
    dZ3 = np.multiply(dA3, np.int64(A3 > 0))
    dW3 = 1./m * np.dot(dZ3, A2.T) + (lambd/m) * W3
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims=True)
    
    # LAYER 2
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) + (lambd/m) * W2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)
    
    # LAYER 1
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T) + (lambd/m) * W1
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {
        "dZ4": dZ4, "dW4": dW4, "db4": db4,
        "dA3": dA3, "dZ3": dZ3, "dW3": dW3, "db3": db3,
        "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
        "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1
    }
    
    return gradients
